In [1]:
from scipy import signal
from scipy.signal import kaiserord, lfilter, firwin, freqz

import librosa
import matplotlib.pyplot as plt

import numpy as np
from ipywidgets import  interactive,    Play,interact_manual,FloatSlider
from matplotlib.ticker import ( FormatStrFormatter )
from IPython.display import (
    Audio, display )
plt.rcParams['axes.grid'] = True
from frequency_waves import get_wave,nice_wave,square,triangle,circle,dirac
import frequency_waves
from Util import fold_and_dft, drop_every_kth, zero_padding, v_to_db ,decimate_in_k

In [2]:
%matplotlib notebook

In [3]:
fs = 44100
t = np.linspace(0,1,fs)
waves_dict = {
    'square':square(t,0,0.001,1),
    'triangle':triangle(t,0.001,0.001,1),
    'circle':circle(t,0.001,0.001,1)*1000,
    'dirac':dirac(t,0,1/fs,0.05)
}

ax_freq = np.arange(0,2*np.pi,2*np.pi/fs)

In [4]:
@interact_manual
def linearity(x1_name = waves_dict.keys(),x2_name = waves_dict.keys(),A=(0,50),B=(0,50)):
    fig,ax=plt.subplots(4,constrained_layout=True)
    fig.tight_layout(pad=0.5)
    fig.suptitle('linearity of '+x1_name+' wave and '+x2_name + ' wave',y= 1)
    x1=waves_dict.get(x1_name)
    x2=waves_dict.get(x2_name)
    ax[0].plot(t,x1,t,x2)
    
    ax[0].set_xlim([0,0.01])
    ax[1].plot(ax_freq,np.fft.fft(x1),ax_freq,np.fft.fft(x2))
    
    ax[2].plot(t,A*x1+B*x2)
    
    ax[2].set_xlim([0,0.01])
    ax[3].plot(ax_freq,np.fft.fft(A*x1+B*x2))
    ax[0].legend([x1_name,x2_name])
    ax[1].legend([x1_name+'_dft',x2_name+'_dft'])
    ax[2].legend([str(A)+'*'+x1_name+'+'+str(B)+'*'+x2_name])
    ax[3].legend([str(A)+'*'+x1_name+'+'+str(B)+'*'+x2_name+'_dft'])

interactive(children=(Dropdown(description='x1_name', options=('square', 'triangle', 'circle', 'dirac'), value…

In [12]:
@interact_manual
def convulotion(x1_name = waves_dict.keys(),x2_name = waves_dict.keys()):
    fig,ax=plt.subplots(4)
    fig.tight_layout(pad=0.5)
    fig.suptitle('convulotion of '+x1_name+' wave and '+x2_name + ' wave',y= 1)
    x1=waves_dict.get(x1_name)
    x2=waves_dict.get(x2_name)
    conv = np.convolve(x1,x2,mode='same')
    ax[0].plot(t,x1,t,x2)
    ax[0].set_xlim([0,0.01])
    ax[1].plot(ax_freq,np.fft.fft(x1),ax_freq,np.fft.fft(x2))
    ax[2].plot(t,conv)
    #ax[2].set_xlim([0,0.01])
    ax[3].plot(ax_freq,np.fft.fft(conv))
    ax[0].legend([x1_name,x2_name])
    ax[1].legend([x1_name+'_dft',x2_name+'_dft'])
    ax[2].legend([x1_name+'$*$'+x2_name])
    ax[3].legend([x1_name+'$*$ '+x2_name+'_dft'])

interactive(children=(Dropdown(description='x1_name', options=('square', 'triangle', 'circle', 'dirac'), value…

In [6]:
@interact_manual
def multipication(x1_name = waves_dict.keys(),x2_name = waves_dict.keys()):
    fig,ax=plt.subplots(4)
    fig.tight_layout(pad=0.5)
    fig.suptitle('multipication of '+x1_name+' wave and '+x2_name + ' wave',y= 1)
    x1=waves_dict.get(x1_name)
    x2=waves_dict.get(x2_name)
    mul = x1*x2
    ax[0].plot(t,x1,t,x2)
    ax[0].set_xlim([0,0.01])
    ax[1].plot(ax_freq,np.fft.fft(x1),ax_freq,np.fft.fft(x2))
    ax[2].plot(t,mul)
    ax[2].set_xlim([0,0.01])
    ax[3].plot(ax_freq,np.fft.fft(mul))
    ax[0].legend([x1_name,x2_name])
    ax[1].legend([x1_name+'_dft',x2_name+'_dft'])
    ax[2].legend([x1_name+'$\times$'+x2_name])
    ax[3].legend([x1_name+'$\times$'+x2_name+'_dft'])

interactive(children=(Dropdown(description='x1_name', options=('square', 'triangle', 'circle', 'dirac'), value…

In [7]:


@interact_manual
def modulation(x1_name = waves_dict.keys(),fi=FloatSlider(min=0,max=np.pi,step = 0.001)):
    fig,ax=plt.subplots(4)
    fig.tight_layout(pad=0.5)
    fig.suptitle('modulation of '+x1_name+' wave and $\phi$',y= 1)
    x1=waves_dict.get(x1_name)
    mod = x1*np.exp(np.array([np.complex(0,fi*x)for x in t]))
    ax[0].plot(t,x1)
    ax[0].set_xlim([0,0.01])
    ax[1].plot(ax_freq,np.fft.fft(x1))
    ax[2].plot(t,mod)
    ax[3].plot(ax_freq,np.fft.fft(mod))
    ax[0].legend([x1_name])
    ax[1].legend([x1_name+'_dft'])
    ax[2].legend([x1_name+'$\cdot e^{j\phi}$'])
    ax[3].legend([x1_name+'$\cdot e^{j\phi}$'+'_dft'])

interactive(children=(Dropdown(description='x1_name', options=('square', 'triangle', 'circle', 'dirac'), value…

In [8]:
c = 3 +6j